In [8]:
import pandas as pd
from datetime import datetime as dt
import math

df = pd.read_csv('export.csv',sep=',',encoding ='latin1', parse_dates=['Completed on', 'Due date', 'Start date'])
df =df[['Id','Assigned to', 'Completed by','Completed on', 'Due date',  'Name', 'Start date', 'Tags', 'Workspaces']]
for col in df.select_dtypes(include=['datetime']):
    df[col] = df[col].dt.strftime('%d-%m-%Y')

df['CompletedWeek'] = pd.DatetimeIndex(df['Completed on']).week
df = df.fillna('NONE')
for col in df.select_dtypes(include=['object']):
    df[col] = df[col].apply(lambda x : "NONE" if x=="NaT" else x)

df['status'] = df.apply(getStatus, axis=1)
df['assignedStatus'] = df.apply(lambda x: "NOT ASSIGNED" if x['Assigned to'] == "NONE" else "ASSIGNED", axis=1)
df['month_year'] = df.apply(setMonthYear, axis=1)
df['month'] = df.apply(setMonth, axis=1)
df['year'] = df.apply(setYear, axis=1)
df['workspaceGroup'] = df.apply(setWorkspace, axis=1)
df = df.sort_values(['month','year'])


noStartDate = df[df['Start date'] == 'NONE']
noStartDateCompleted = noStartDate[(noStartDate['status']=='CLOSED')]
noStartDateOpen = noStartDate[(noStartDate['status']=='OPEN')]
withStartDate = df[df['Start date'] != 'NONE']
projectedTasks = withStartDate[pd.to_datetime(withStartDate['Start date'], errors='ignore') > dt.now()]
projectedTasksWoBl = projectedTasks[~projectedTasks['Tags'].str.contains('status')]
overdueYetToComplete = withStartDate[withStartDate['status']=='OVERDUE_OPEN']
overdueYetToCompleteWoBl = overdueYetToComplete[~overdueYetToComplete['Tags'].str.contains('status')]
overdueComplete = withStartDate[withStartDate['status']=='OVERDUE_CLOSED']
overdueCompleteWoBl = overdueComplete[~overdueComplete['Tags'].str.contains('status')]
openTasks = withStartDate[withStartDate['status'].str.contains('OPEN')]
openTasks = openTasks[pd.to_datetime(openTasks['Start date'], errors='ignore') < dt.now()]
openTasksWoBl = openTasks[~openTasks['Tags'].str.contains('status')]
openTasksWoBl = openTasksWoBl[~openTasksWoBl['Tags'].str.contains('Bug')]
closeTasks = withStartDate[withStartDate['status'].str.contains('CLOSED')]
closeTasks = closeTasks[pd.to_datetime(closeTasks['Start date'], errors='ignore') < dt.now()]
closeTasksWoBl = closeTasks[~closeTasks['Tags'].str.contains('status')]
closeTasksWoBl = closeTasksWoBl[~closeTasksWoBl['Tags'].str.contains('Bug')]


completeOpenList = pd.concat([projectedTasksWoBl,overdueYetToCompleteWoBl,openTasksWoBl]).drop_duplicates().reset_index(drop=True)







In [7]:
import calendar
import datetime
import pandas as pd



def getStatus(x):
    """ status information """
    status = ""
    if x['Completed on'] == "NONE":
        if x['Due date'] != "NONE":
            d3 = pd.to_datetime(x['Due date']) - datetime.datetime.now()
            if d3.days < 0:
                status = "OVERDUE"
            else:
                status = "OPEN"
        elif x['Due date'] == "NONE":
            status = "OPEN"
    elif x['Completed on'] != "NONE":
        if x['Due date'] != "NONE":
            d3 = pd.to_datetime(x['Completed on']) - pd.to_datetime(x['Due date'])
            if d3.days > 0:
                status = "OVERDUE_CLOSED"
            else:
                status = "CLOSED"
        elif x['Due date'] == "NONE":
            status = "CLOSED"
    return status

def setMonthYear(row):
    """ month-year"""
    dateArr = []
    if row['Start date'] != "NONE":
        d3 = pd.to_datetime(row['Start date'])
        dateArr.append(str(calendar.month_abbr[d3.month]))
        dateArr.append(str(d3.year))
    return "-".join(dateArr)


def setYear(row):
    """year"""
    if row['Start date'] != "NONE":
        d5 = pd.to_datetime(row['Start date'])
        return d5.year


def setMonth(x):
    """month"""
    if x['Start date'] != "NONE":
        d1 = pd.to_datetime(x['Start date'])
        return d1.month


def setWorkspace(row):
    """workspace data"""
    workspace = ""
    if isinstance(row['Workspaces'], str):
        if 'operational' in row['Workspaces']:
            workspace = "Operational"
        elif 'projects' in row['Workspaces']:
            workspace = "Project"
        else:
            workspace = "None"
    else:
        workspace = "None"
    return workspace

def dfDateFilter(start_date,end_date, df):
    if start_date is not None and end_date is not None:
        df1 = df[(pd.to_datetime(df['Start date'], dayfirst=True, errors='ignore') > start_date)&(pd.to_datetime(df['Start date'], dayfirst=True, errors='ignore') < end_date)]
    elif start_date is not None and end_date is None:
        df1 = df[(pd.to_datetime(df['Start date'], dayfirst=True, errors='ignore') > start_date)]
    elif start_date is None and end_date is not None:
        df1 = df[(pd.to_datetime(df['Start date'], dayfirst=True, errors='ignore') < end_date)]
    else:
        df1 = df
    return df1

def groupDf(df):
    groupedByTagsWs = df
    groupedByTagsWs['startDateWeek'] = pd.DatetimeIndex(groupedByTagsWs['Start date']).week
    groupedByTagsWs1 = pd.DataFrame({'count' : groupedByTagsWs.groupby(['Tags','startDateWeek']).size()}).reset_index()
    groupedByTagsWs1 = groupedByTagsWs1.pivot(index='startDateWeek',columns='Tags',values='count')
    return groupedByTagsWs1

In [9]:
df.head()

Id    Assigned to             Completed by Completed on    Due date  \
2302   254   Shini Sharma          Joerg Engelmann   17-07-2018        NONE   
957   4401    Kiran Kumar  Sakthi Mainthan Abishek   15-12-2017  01-05-2018   
960   4826  Ipsita Bhatta            Ipsita Bhatta   01-05-2018  01-05-2018   
961   4854  Ipsita Bhatta            Ipsita Bhatta   01-05-2018  01-05-2018   
962   4881  Ipsita Bhatta            Ipsita Bhatta   01-05-2018  01-05-2018   

                                                   Name  Start date  \
2302                                MAXIMO operator fix  01-01-2017   
957   Provide completed Rotable sheet for processing...  01-01-2018   
960              SB Embodiment and SB Campaign activity  01-01-2018   
961                              Delta Event Management  01-01-2018   
962          New aircraft Creation and Missing Operator  01-01-2018   

             Tags                      Workspaces CompletedWeek  status  \
2302       MAXIMO              projects/dev tasks            29  CLOSED   
957   MAXIMO - SE  operational process/#3 monthly            50  CLOSED   
960            SE   operational process/#2 weekly             1  CLOSED   
961            SE   operational process/#2 weekly             1  CLOSED   
962            SE   operational process/#2 weekly             1  CLOSED   

     assignedStatus month_year  month    year workspaceGroup  
2302       ASSIGNED   Jan-2017    1.0  2017.0        Project  
957        ASSIGNED   Jan-2018    1.0  2018.0    Operational  
960        ASSIGNED   Jan-2018    1.0  2018.0    Operational  
961        ASSIGNED   Jan-2018    1.0  2018.0    Operational  
962        ASSIGNED   Jan-2018    1.0  2018.0    Operational

In [14]:
tagstatus = df.groupby([ 'Tags', 'status'])['Id'].count().reset_index().rename(
    columns={'Id': 'count'}).sort_values('count')

In [24]:
tagstatus

Tags          status  count
0        ##BackEnd - Data Engineering and Science         OVERDUE      1
41          Data Engineering and Science - MAXIMO            OPEN      1
44  Data Engineering and Science - status-backlog         OVERDUE      1
50                                    IT - MAXIMO            OPEN      1
29                              COGNOS - xmas2017          CLOSED      1
..                                            ...             ...    ...
45                                             IT          CLOSED    257
66                                         MAXIMO          CLOSED    519
81                                             SE  OVERDUE_CLOSED    575
69                                         MAXIMO  OVERDUE_CLOSED    919
78                                             SE          CLOSED   1239

[92 rows x 3 columns]

In [19]:
tagstatus['Tags'].unique()

array(['##BackEnd - Data Engineering and Science',
       'Data Engineering and Science - MAXIMO',
       'Data Engineering and Science - status-backlog', 'IT - MAXIMO',
       'COGNOS - xmas2017', 'IT - MAXIMO - xmas2017', 'IT - SE',
       'IT - status-backlog', 'COGNOS - SE - status-on hold',
       'COGNOS - MAXIMO - status-on hold',
       'IT - status-backlog - status-on hold', 'IT - status-blocked',
       'IT - status-on hold', 'MAXIMO - SE', 'COGNOS - IT',
       'MAXIMO - SE - status-backlog', 'SE - xmas2017',
       '#BizAv - #CLE - COGNOS - Data Engineering and Science - IT',
       'status-backlog', 'status-blocked', 'COGNOS - MAXIMO', 'Bug - IT',
       '#CLE - MAXIMO', '#CLE - COGNOS', 'status-on hold',
       'Data Engineering and Science - IT - MAXIMO',
       'COGNOS - status-blocked', 'COGNOS - status-on hold',
       'COGNOS - MAXIMO - SE', 'COGNOS',
       'Data Engineering and Science - IT', 'COGNOS - status-backlog',
       'SE - status-backlog', 'COGNOS - IT - M

In [20]:
tagstatus['count'].unique()

array([   1,    2,    3,    4,    5,    6,    8,    9,   10,   11,   14,
         15,   16,   19,   23,   26,   27,   28,   30,   42,   53,   59,
         62,   73,   84,  102,  165,  168,  184,  226,  231,  257,  519,
        575,  919, 1239], dtype=int64)

In [25]:
import plotly.graph_objs as go
fig = go.Figure()

tagstatus = df.groupby([ 'Tags', 'status'])['Id'].count().reset_index().rename(
    columns={'Id': 'count'}).sort_values('count')

fig.add_trace(go.Bar(
    x=tagstatus['count'].unique(),
    y=tagstatus['Tags'].unique(),
    name='SF Zoo',
    orientation='h',

))


fig.update_layout(barmode='stack')
fig.show()

In [26]:
groupTags = pd.DataFrame({'count' : df.groupby( [ "Tags", "status"] ).size()}).reset_index()
tagNames = (groupTags['Tags'].unique().tolist())
#print(groupTags)

groupTags = groupTags.pivot(index='Tags',columns='status',values='count')
groupTags = groupTags.fillna(0)
groupTags

status                                              CLOSED  OPEN  OVERDUE  \
Tags                                                                        
##BackEnd - Data Engineering and Science               0.0   0.0      1.0   
#BizAv - #CLE - COGNOS - Data Engineering and S...     0.0   1.0      0.0   
#CLE - COGNOS                                          0.0   9.0      1.0   
#CLE - MAXIMO                                          0.0   1.0      0.0   
Bug - IT                                               1.0   2.0      0.0   
COGNOS                                               231.0  28.0      2.0   
COGNOS - IT                                            1.0   0.0      0.0   
COGNOS - IT - MAXIMO                                   6.0   0.0      0.0   
COGNOS - MAXIMO                                       11.0   1.0      0.0   
COGNOS - MAXIMO - SE                                   2.0   0.0      0.0   
COGNOS - MAXIMO - status-on hold                       5.0   0.0      0.0   
COGNOS - SE                                            6.0   0.0      0.0   
COGNOS - SE - status-on hold                           0.0   1.0      0.0   
COGNOS - status-backlog                                3.0   6.0      0.0   
COGNOS - status-blocked                                0.0   2.0      0.0   
COGNOS - status-on hold                                0.0   2.0      0.0   
COGNOS - xmas2017                                      1.0   0.0      0.0   
Data Engineering and Science                          84.0  53.0      9.0   
Data Engineering and Science - IT                     11.0  10.0      3.0   
Data Engineering and Science - IT - MAXIMO             3.0   2.0      0.0   
Data Engineering and Science - MAXIMO                  1.0   1.0      0.0   
Data Engineering and Science - status-backlog          2.0  15.0      1.0   
IT                                                   257.0  59.0     26.0   
IT - MAXIMO                                           10.0   1.0      0.0   
IT - MAXIMO - xmas2017                                 0.0   0.0      0.0   
IT - SE                                                2.0   1.0      1.0   
IT - status-backlog                                   16.0  42.0      1.0   
IT - status-backlog - status-on hold                   0.0   1.0      0.0   
IT - status-blocked                                    1.0   0.0      0.0   
IT - status-on hold                                    1.0   0.0      0.0   
MAXIMO                                               519.0  62.0    184.0   
MAXIMO - SE                                           23.0   1.0      0.0   
MAXIMO - SE - status-backlog                           0.0   1.0      0.0   
NONE                                                 226.0  73.0     30.0   
SE                                                  1239.0  42.0     11.0   
SE - status-backlog                                    0.0   3.0      0.0   
SE - xmas2017                                          1.0   0.0      0.0   
status-backlog                                         2.0   9.0      1.0   
status-blocked                                         1.0   0.0      0.0   
status-on hold                                         2.0   1.0      3.0   

status                                              OVERDUE_CLOSED  
Tags                                                                
##BackEnd - Data Engineering and Science                       0.0  
#BizAv - #CLE - COGNOS - Data Engineering and S...             0.0  
#CLE - COGNOS                                                  0.0  
#CLE - MAXIMO                                                  0.0  
Bug - IT                                                       3.0  
COGNOS                                                       165.0  
COGNOS - IT                                                    1.0  
COGNOS - IT - MAXIMO                                           0.0  
COGNOS - MAXIMO                                                9.0  
COGNOS